In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from IPython.display import display, Math

In [ ]:
def RectInsert(nome, X, Xspan, Y, Yspan, Z, Zspan, material, lumerical):
    
    lumerical.switchtolayout()
    lumerical.addrect()
    lumerical.set({"name": nome,
        "x":X,"y":Y,"z":Z,
        "x span":Xspan, "y span":Yspan, "z span":Zspan,
        "material":material
        })

In [ ]:

nomes = [["MMImeio", "Entrada", "Saida1", "Saida2", "Saida3", "EntradaTP", "Saida1TP", "Saida2TP", "Saida3TP"], ["mash"]]
X = [[][]]

In [ ]:
# Body config------------------------------------
BdX_Span = 31.87e-6 # Body X Span
BdY_Span = 6e-6 # Body y Span


# Ports config-----------------------------------

OutMx = [1.6e-6,-1.6e-6] # Array of output ports y positions (put 0 on first element to ignore this line)
InMx = [0]               # Array of input ports y positions (put 0 on first element to ignore this line) 

# If the lines above has to be ignored, use these ones instead: 
InP = 1 # Number of Input ports
OutP = 2 # Number of Output ports
Asy = 0 # Force asymmetry on inputs ports auto-displacement (0 -> Symetry, 1 -> Asymmetry)
         
PYSpan_in = 12e-6 # Input Port Y Span 
PYSpan_out = 3e-6 # Output Port Y Span 


# Tappers config---------------------------------
SetToTappers = 1 # Set tappers as inputs/outputs
TLen = 4e-6 # Tapper Length
TWidth = 1.1e-6 # Tapper Width


# Mesh config------------------------------------
MeshEnableIn = 0 # enable input mesh insertion
MeshEnableOut = 1 # enable output mesh insertion
MeshType = 1 # 0 for Override, 1 for Multipler

MeshOver = [[0.05e-6,0.05e-6,0.05e-6] , [0.05e-6,0.05e-6,0.05e-6]] # Override mesh value dx,dy and dz ([[Input Mesh],[Output Mesh]])
MeshMultiplier = [[2,2,2] , [4,4,4]] # Mesh multipler value dx,dy and dz ([[Input Mesh],[Output Mesh]])


# EME config-------------------------------------
EME_MC = 100 # Number of EME Mesh Cells (Y and Z)
SideCells = 4 # Number of Cells on Input/Output groups